In [4]:
import pandas as pd
import math
from statsmodels.tsa.stattools import adfuller

# Load the dataset
data = pd.read_csv("TFTP_mini.csv")

ddos_detected_true = 0
ddos_detected_false = 0
ddos_true = 24
ddos_false = 20
degrees_of_ddos = []
def calculate_entropy(data, normalized=True):
    frequency = data.value_counts()
    entropy = 0.0
    normalized_ent = 0.0
    n = 0

    if len(frequency) <= 1:
        return None, None

    for i, x in list(enumerate(frequency.index)):
        try:
            p_x = frequency[x] / sum(frequency)
            if p_x > 0:
                n += 1
                entropy += -p_x * math.log(p_x, 2)
        except KeyError:
            continue

    if normalized:
        if math.log(n) > 0:
            normalized_ent = entropy / math.log(n, 2)
            return entropy, normalized_ent
    else:
        return entropy



# Select relevant columns for analysis
columns = ["Timestamp", "Total Length of Fwd Packets", "Total Length of Bwd Packets", "Flow Duration"]
selected_data = data[columns]

# Set the window size
window_size = 5000

# Split the data into windows
windows = [selected_data[i : i + window_size] for i in range(0, len(selected_data), window_size)]

for window in windows:
    
        # Additional steps for entropy calculation and detection
    flow_size = window["Total Length of Fwd Packets"] + window["Total Length of Bwd Packets"]
    flow_rate = flow_size / window["Flow Duration"]
    flow_duration = window["Flow Duration"]
    flow_length = window["Total Length of Fwd Packets"] + window["Total Length of Bwd Packets"]

        # Calculate entropy for each parameter
    entropy_flow_size, normalized_entropy_flow_size = calculate_entropy(flow_size)
    entropy_flow_rate, normalized_entropy_flow_rate = calculate_entropy(flow_rate)
    entropy_flow_duration, normalized_entropy_flow_duration = calculate_entropy(flow_duration)
    entropy_flow_length, normalized_entropy_flow_length = calculate_entropy(flow_length)

    if entropy_flow_size is not None:
        print("Entropy (Flow Size):", entropy_flow_size)
        print("Normalized Entropy (Flow Size):", normalized_entropy_flow_size)
    else:
        print("Entropy (Flow Size): Not enough data")
        
    if entropy_flow_rate is not None:
        print("Entropy (Flow Rate):", entropy_flow_rate)
        print("Normalized Entropy (Flow Rate):", normalized_entropy_flow_rate)
    else:
        print("Entropy (Flow Rate): Not enough data")
        


    if entropy_flow_duration is not None:
        print("Entropy (Flow Duration):", entropy_flow_duration)
        print("Normalized Entropy (Flow Duration):", normalized_entropy_flow_duration)
    else:
        print("Entropy (Flow Duration): Not enough data")
        
    if entropy_flow_length is not None:
        print("Entropy (Flow Length):", entropy_flow_length)
        print("Normalized Entropy (Flow Length):", normalized_entropy_flow_length)
    else:
        print("Entropy (Flow Length): Not enough data")

        # Calculate degree of DDoS attack
        # Calculate degree of DDoS attack
    entropy_values = [
    normalized_entropy_flow_size,
    normalized_entropy_flow_rate,
    normalized_entropy_flow_duration,
    normalized_entropy_flow_length,
    ]

    valid_entropy_values = [val for val in entropy_values if val is not None]

    if valid_entropy_values:
        degree_of_ddos = sum(valid_entropy_values) / len(valid_entropy_values)
        print("Degree of DDoS Attack:", degree_of_ddos)
        degrees_of_ddos.append(degree_of_ddos)
        if degree_of_ddos > 0.5:
            print("DDoS attack detected.")
            ddos_detected_true = ddos_detected_true + 1
        else:
            print("No DDoS attack detected.")
            ddos_detected_false = ddos_detected_false + 1
    else:
        print("Insufficient data for entropy calculation.")
            
print("True ", ddos_detected_true)
print("False", ddos_detected_false)
accuracy = (ddos_detected_true/ddos_true + ddos_detected_false/ddos_false)/2



C:\Users\DELL\AppData\Local\Temp\ipykernel_14628\2808958253.py:6: DtypeWarning: Columns (85) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("TFTP_mini.csv")


Entropy (Flow Size): 2.5142055806399983
Normalized Entropy (Flow Size): 0.563795074271883
Entropy (Flow Rate): 9.192908736105084
Normalized Entropy (Flow Rate): 0.7958713758436937
Entropy (Flow Duration): 9.28192003631179
Normalized Entropy (Flow Duration): 0.8010419204258735
Entropy (Flow Length): 2.5142055806399983
Normalized Entropy (Flow Length): 0.563795074271883
Degree of DDoS Attack: 0.6811258612033333
DDoS attack detected.
Entropy (Flow Size): 2.5219185441170993
Normalized Entropy (Flow Size): 0.5835170064722757
Entropy (Flow Rate): 9.241738109947365
Normalized Entropy (Flow Rate): 0.800399115653764
Entropy (Flow Duration): 9.275927176383952
Normalized Entropy (Flow Duration): 0.8009469407568246
Entropy (Flow Length): 2.5219185441170993
Normalized Entropy (Flow Length): 0.5835170064722757
Degree of DDoS Attack: 0.692095017338785
DDoS attack detected.
Entropy (Flow Size): 1.4768559700949018
Normalized Entropy (Flow Size): 0.3613136137142563
Entropy (Flow Rate): 9.687358508957775

In [6]:
print("Accuracy: ", round(accuracy,2)*100,"%")

Accuracy:  94.0 %
